In [1]:
import numpy as np
import scipy as sp

from scipy import integrate
from scipy.optimize import fmin_powell, fminbound, brentq

from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

# Functions to Synthesize Data

In [2]:
# Functions to create synthetic data
def irt_evaluation(difficulty, discrimination, thetas):
    """
        Evaluates an IRT model and returns the exact values
        
        Args:
            difficulty: [array] of difficulty parameters
            discrimination:  [array | number] of discrimination parameters
            thetas: [array] of person abilities
            
        Returns:
            dichotomous matrix of [difficulty.size x thetas.size] representing
            synthetic data
    """
    # If discrimination is a scalar, make it an array
    if not np.ndim(discrimination):
        discrimination = np.ones_like(difficulty) * discrimination

    kernel = difficulty[:, None] - thetas
    kernel *= discrimination[:, None]
    return 1.0 / (1 + np.exp(kernel))



def create_synthetic_irt_dichotomous(difficulty, discrimination, thetas):
    """
        Creates synthetic IRT data to test parameters estimation
        functions.  Only for use with dichotomous outputs
        
        Args:
            difficulty: [array] of difficulty parameters
            discrimination:  [array | number] of discrimination parameters
            thetas: [array] of person abilities
            
        Returns:
            dichotomous matrix of [difficulty.size x thetas.size] representing
            synthetic data
    """
    continuous_output = irt_evaluation(difficulty, discrimination, thetas)

    # convert to binary based on probability
    random_compare = np.random.rand(*continuous_output.shape)
    
    return random_compare <= continuous_output


# Utility functions

In [3]:
from scipy.special import roots_legendre


def _get_quadrature_points(n, a, b):
    """
        Utility function to get the legendre points, 
        shifted from [-1, 1] to [a, b]
        
        Args:
            n: number of quadrature_points
            a: lower bound of integration
            b: upper bound of integration 
            
        A local function of the based fixed_quad found in scipy
    """
    x, w = roots_legendre(n)
    x = np.real(x)
    
    return (b - a) * (x + 1) * 0.5 + a
    

def _compute_partial_integral(theta, difficulty, discrimination, the_sign):
    """
        To be added
    """
    if np.ndim(discrimination) < 1:
        discrimination = np.full(the_sign.shape[0], discrimination)
        
    kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
    kernel *= discrimination[:, None, None]   
    kernel *= (theta[None, None, :] - difficulty[:, None, None])
    
    # Distribution
    gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)

    return  gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()    

## Conditional Probability Rauch

In [542]:
def conditional_probability(dataset, discrimination=1, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            max_iter: maximum number of iterations to run
            
        Returns:
            array of discrimination estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # First run mml to get coarse guess,
    # use the mean to set identify the solution
    betas = rauch_estimate(dataset, discrimination) * 0
    identifying_mean = betas.mean()
    betas -= identifying_mean

    # Remove the zero and full count values
    if(unique_sets[:, 0].sum() == 0):
        unique_sets = np.delete(unique_sets, 0, axis=1)
        counts = np.delete(counts, 0)

    if(unique_sets[:, -1].sum() == n_items):
        unique_sets = np.delete(unique_sets, -1, axis=1)
        counts = np.delete(counts, -1)

    response_set_sums = unique_sets.sum(axis=0)

    def _denominator(betas):
        """Computes the symmetric functions based on the betas
        
         Indexes by score, left to right
        
        """
        polynomials = np.c_[np.ones_like(betas), np.exp(-betas)]
        otpt = 1
        
        for polynomial in polynomials:
            otpt = np.convolve(otpt, polynomial)
        return otpt
 
    for iteration in range(max_iter):
        previous_betas = betas.copy()
        
        for ndx in range(n_items):
            partial_conv = _denominator(np.delete(betas, ndx))
            
            def min_func(estimate):
                betas[ndx] = estimate
                full_convolution = np.convolve([1, np.exp(-estimate)], partial_conv)
                
                numerator = np.exp(-np.sum(unique_sets * betas[:,None], axis=0))
                denominator = full_convolution[response_set_sums]
                
                return -np.log(numerator / denominator).dot(counts)
            
            betas[ndx] = fminbound(min_func, -5, 5)

            # recenter
            betas += (identifying_mean - betas.mean())
        
        if np.abs(betas - previous_betas).max() < 1e-3:
            print(f'Ended in {iteration} iterations')
            break
            
    return betas / discrimination


# Estimate functions based on approximation

In [4]:
def rauch_estimate(dataset, discrimination=1):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            
        Returns:
            array of discrimination estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    return (np.sqrt(1 + discrimination**2 / 3) * 
            np.log(n_no / n_yes) / discrimination)


def onepl_estimate(dataset):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)

    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
        kernel *= discrimination   
        kernel *= (theta[None, None, :] - difficulty[:, None, None])
        
        return  gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = np.sqrt(1 + estimate**2 / 3) * scalar / estimate
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (difficulty, estimate, unique_sets), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    discrimination = fminbound(min_func, 0.25, 10)
    
    return discrimination, np.sqrt(1 + discrimination**2 / 3) * scalar / discrimination


def twopl_estimate(dataset, max_iter=25):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            max_iter:  maximum number of iterations to run
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets
    
    theta = _get_quadrature_points(61, -5, 5)

    # Inline definition of quadrature function
    def quadrature_function(theta, discrimination, old_discrimination, 
                            difficulty, old_difficulty,
                            partial_int, the_sign):
        kernel1 = the_sign[:, None] * (theta[None, :] - difficulty)
        kernel1 *= discrimination

        kernel2 = the_sign[:, None] * (theta[None, :] - old_difficulty)
        kernel2 *= old_discrimination

        return partial_int * (1 + np.exp(kernel2)) / (1 + np.exp(kernel1))
    
    
    # Inline definition of cost function to minimize
    def min_func(estimate, dataset, old_estimate, old_difficulty,
                 partial_int, the_sign):
        new_difficulty = rauch_estimate(dataset, estimate)
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (estimate, old_estimate, 
                                     new_difficulty, old_difficulty,
                                     partial_int, the_sign), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    initial_guess = np.ones((dataset.shape[0],))
    difficulties = rauch_estimate(dataset)
    
    for iteration in range(max_iter):
        previous_guess = initial_guess.copy()
        previous_difficulty = difficulties.copy()

        #Quadrature evaluation for values that do not change
        partial_int = _compute_partial_integral(theta, difficulties,
                          initial_guess, the_sign)
        
        for ndx in range(n_items):
            def min_func_local(estimate):
                return min_func(estimate, dataset[ndx].reshape(1, -1),  
                                previous_guess[ndx], 
                                previous_difficulty[ndx],
                                partial_int, the_sign[ndx])

            initial_guess[ndx] = fminbound(min_func_local, 0.25, 6, xtol=1e-3)
            difficulties[ndx] = rauch_estimate(dataset[ndx].reshape(1, -1), 
                                               initial_guess[ndx])
            
            partial_int = quadrature_function(theta, initial_guess[ndx], 
                                              previous_guess[ndx], difficulties[ndx],
                                              previous_difficulty[ndx],
                                              partial_int, the_sign[ndx])            

        if np.abs(initial_guess - previous_guess).max() < 1e-3:
            break
            
    return initial_guess, difficulties


# Functions based on full integral

In [5]:
def rauch_estimate_int(dataset, discrimination=1, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            max_iter: maximum number of iterations to run
            
        Returns:
            array of discrimination estimates
    """
    n_items = dataset.shape[0]
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = n_yes / (n_yes + n_no)
    
    if np.ndim(discrimination) < 1:
        discrimination = np.full(n_items, discrimination)
   
    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination):
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        return irt_evaluation(np.array([difficulty]), np.array([discrimination]), theta) * gauss

    the_parameters = np.zeros((n_items,))

    # Perform the minimization
    for ndx in range(n_items):
        
        # Minimize each item separately
        def min_zero_local(estimate):
            return (scalar[ndx] - 
                    integrate.fixed_quad(quadrature_function, -10, 10, 
                    (estimate, discrimination[ndx]), n=101)[0])
        
        the_parameters[ndx] = brentq(min_zero_local, -6, 6)
            
    return the_parameters


def onepl_estimate_int(dataset):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
        kernel *= discrimination   
        kernel *= (theta[None, None, :] - difficulty[:, None, None])
        
        return  gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = rauch_estimate_int(dataset, estimate)
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (difficulty, estimate, unique_sets), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    discrimination = fminbound(min_func, 0.25, 10)
    
    return discrimination, rauch_estimate_int(dataset, discrimination)


def twopl_estimate_int(dataset, max_iter=25):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            max_iter:  maximum number of iterations to run
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets
    
    theta = _get_quadrature_points(61, -5, 5)

    # Inline definition of quadrature function
    def quadrature_function(theta, discrimination, old_discrimination, 
                            difficulty, old_difficulty,
                            partial_int, the_sign):
        kernel1 = the_sign[:, None] * (theta[None, :] - difficulty)
        kernel1 *= discrimination

        kernel2 = the_sign[:, None] * (theta[None, :] - old_difficulty)
        kernel2 *= old_discrimination

        return partial_int * (1 + np.exp(kernel2)) / (1 + np.exp(kernel1))
    
    
    # Inline definition of cost function to minimize
    def min_func(estimate, dataset, old_estimate, old_difficulty,
                 partial_int, the_sign):
        new_difficulty = rauch_estimate_int(dataset, estimate)
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (estimate, old_estimate, 
                                     new_difficulty, old_difficulty,
                                     partial_int, the_sign), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    initial_guess = np.ones((dataset.shape[0],))
    difficulties = rauch_estimate(dataset)
    
    for iteration in range(max_iter):
        previous_guess = initial_guess.copy()
        previous_difficulty = difficulties.copy()

        #Quadrature evaluation for values that do not change
        partial_int = _compute_partial_integral(theta, difficulties,
                          initial_guess, the_sign)
        
        for ndx in range(n_items):
            def min_func_local(estimate):
                return min_func(estimate, dataset[ndx].reshape(1, -1),  
                                previous_guess[ndx], 
                                previous_difficulty[ndx],
                                partial_int, the_sign[ndx])

            initial_guess[ndx] = fminbound(min_func_local, 0.25, 6, xtol=1e-3)
            difficulties[ndx] = rauch_estimate_int(dataset[ndx].reshape(1, -1), 
                                                   initial_guess[ndx])
            
            partial_int = quadrature_function(theta, initial_guess[ndx], 
                                              previous_guess[ndx], difficulties[ndx],
                                              previous_difficulty[ndx],
                                              partial_int, the_sign[ndx])            

        if np.abs(initial_guess - previous_guess).max() < 1e-3:
            break
            
    return initial_guess, difficulties


# Functions based on complete joint probability

In [6]:
def _rauch_estimate_full_abstract(dataset, discrimination=1, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            max_iter: maximum number of iterations to run
            
        Returns:
            array of discrimination estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets

    theta = _get_quadrature_points(61, -5, 5)
    
    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, old_difficulty, partial_int, the_sign):
        kernel1 = the_sign[:, None] * (theta[None, :] - difficulty)
        kernel1 *= discrimination

        kernel2 = the_sign[:, None] * (theta[None, :] - old_difficulty)
        kernel2 *= discrimination

        return partial_int * (1 + np.exp(kernel2)) / (1 + np.exp(kernel1))
    
    # Inline definition of cost function to minimize
    def min_func(difficulty, old_difficulty, partial_int, the_sign):
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                (difficulty, old_difficulty, partial_int, the_sign), n=61)[0] + 1e-23
        return -np.log(otpt).dot(counts)

    # Get approximate guess to begin with
    initial_guess = rauch_estimate(dataset, discrimination=discrimination)

    for iteration in range(max_iter):
        previous_guess = initial_guess.copy()

        #Quadrature evaluation for values that do not change
        partial_int = _compute_partial_integral(theta, initial_guess,
                          discrimination, the_sign)
                
        for ndx in range(n_items):
            # Minimize each one separately
            value = initial_guess[ndx] * 1.0
            
            def min_func_local(estimate):
                return min_func(estimate, previous_guess[ndx], 
                                partial_int, the_sign[ndx])
            
            initial_guess[ndx] = fminbound(min_func_local, 
                                           value-0.75,
                                           value+0.75)
            
            partial_int = quadrature_function(theta, initial_guess[ndx], 
                                              previous_guess[ndx], partial_int, the_sign[ndx])

        if(np.abs(initial_guess - previous_guess).max() < 0.001):
            break
            
    # Get the value of the cost function
    cost = integrate.fixed_quad(lambda x: partial_int, -5, 5, n=61)[0]
    
    return initial_guess, -np.log(cost).dot(counts)


def rauch_estimate_full(dataset, discrimination=1, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            max_iter: maximum number of iterations to run
            
        Returns:
            array of discrimination estimates
    """
    return _rauch_estimate_full_abstract(dataset, discrimination, max_iter)[0]


def onepl_estimate_full(dataset, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    def min_func_local(estimate):
        _, cost = _rauch_estimate_full_abstract(dataset, estimate, max_iter)
        return cost
    
    discrimination = fminbound(min_func_local, 0.5, 4)
    
    return discrimination, rauch_estimate_full(dataset, discrimination)


def twopl_estimate_full(dataset, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = (-1)**unique_sets

    theta = _get_quadrature_points(61, -5, 5)
    
    # Inline definition of quadrature function
    def quadrature_function(theta, estimates, old_estimates, partial_int, the_sign):
        kernel1 = the_sign[:, None] * (theta[None, :] - estimates[1])
        kernel1 *= estimates[0]

        kernel2 = the_sign[:, None] * (theta[None, :] - old_estimates[1])
        kernel2 *= old_estimates[0]

        return partial_int * (1 + np.exp(kernel2)) / (1 + np.exp(kernel1))
    
    # Inline definition of cost function to minimize
    def min_func(estimates, old_estimates, partial_int, the_sign):
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                (estimates, old_estimates, partial_int, the_sign), n=61)[0] + 1e-23
        return -np.log(otpt).dot(counts)

    # Get approximate guess to begin with rasch model
    a1, b1 = twopl_estimate(dataset)
    initial_guess = np.c_[a1, b1]

    for iteration in range(max_iter):
        previous_guess = initial_guess.copy()

        #Quadrature evaluation for values that do not change
        partial_int = _compute_partial_integral(theta, initial_guess[:, 1],
                          initial_guess[:, 0], the_sign)
                
        for ndx in range(n_items):
            # Minimize each one separately
            value = initial_guess[ndx] * 1.0
            
            def min_func_local(estimate):
                return min_func(estimate, previous_guess[ndx], 
                                partial_int, the_sign[ndx])

            initial_guess[ndx] = fmin_powell(min_func_local, value, xtol=1e-3, disp=0)
            partial_int = quadrature_function(theta, initial_guess[ndx], 
                                              previous_guess[ndx], partial_int, the_sign[ndx])

        if(np.abs(initial_guess - previous_guess).max() < 0.001):
            break
                
    return initial_guess[:, 0], initial_guess[:, 1]



# Create a set of synthetic data

In [544]:
n_items, n_participants = 100, 2000
diffc = np.linspace(-2.5, 2.5, n_items)
discr = 1.26#1.0 + np.random.rand(n_items,)
thetas = np.random.randn(n_participants)

syn_data = create_synthetic_irt_dichotomous(diffc, discr, thetas)

In [8]:
b_est = rauch_estimate(syn_data, 1.0)
b_int = rauch_estimate_int(syn_data, 1.0)
b_full = rauch_estimate_full(syn_data, 1.0)

In [9]:
a_est = onepl_estimate(syn_data)
a_int = onepl_estimate_int(syn_data)
a_full = onepl_estimate_full(syn_data)

In [68]:
c_est = twopl_estimate(syn_data)
c_int = twopl_estimate_int(syn_data)
c_full = twopl_estimate_full(syn_data)

In [69]:
np.sqrt(np.square(c_est[0] - discr).mean()), np.sqrt(np.square(c_est[1] - diffc).mean())

(0.14253848245771353, 0.13211132771089557)

In [70]:
np.sqrt(np.square(c_int[0] - discr).mean()), np.sqrt(np.square(c_int[1] - diffc).mean())

(0.1326282901494772, 0.11324222526558021)

In [71]:
np.sqrt(np.square(c_full[0] - discr).mean()), np.sqrt(np.square(c_full[1] - diffc).mean())

(0.13417876733982936, 0.11486588565924734)

In [93]:
complex(4, 3).real

4.0

In [545]:
conditional_probability(syn_data, 1.26)

Ended in 5 iterations


array([-2.44665315, -2.37710512, -2.37710663, -2.21659607, -2.33491451,
       -2.32900824, -2.17957588, -2.0842642 , -2.09884573, -2.01828887,
       -1.94670522, -1.88701747, -1.97310095, -1.89958786, -1.79806248,
       -1.7670296 , -1.84183657, -1.68125726, -1.62780363, -1.53921145,
       -1.48055269, -1.35390771, -1.41764327, -1.30677368, -1.28363769,
       -1.19097865, -1.21860568, -1.19097245, -1.09176345, -1.0734858 ,
       -1.04249018, -1.02969454, -0.89077256, -0.8381402 , -0.76094545,
       -0.72877062, -0.61408507, -0.6563777 , -0.60082395, -0.51135818,
       -0.47264454, -0.4003204 , -0.39398402, -0.33934767, -0.25819571,
       -0.24578317, -0.18190266, -0.12040419, -0.08974328, -0.1020026 ,
        0.05910308,  0.02037843,  0.04279489,  0.21862969,  0.17348551,
        0.27634004,  0.30325418,  0.33025655,  0.47543265,  0.48609629,
        0.56575638,  0.61821115,  0.70274285,  0.65134755,  0.74349728,
        0.82659039,  0.81491317,  0.95587033,  0.9145028 ,  0.96

In [546]:
vv = rauch_estimate(syn_data, 1.26)
vv.mean()

-0.00016840622602676714

In [507]:
    def _denominator(betas):
        """Computes the symmetric functions based on the betas
        
         Indexes by score, left to right
        
        """
        new_length = betas.shape[0] + 1
        polynomials = np.c_[np.ones_like(betas), np.exp(-betas)]
        spectrum = np.fft.fft(polynomials, n=new_length, norm=None, axis=1)
        otpt = 1
        
        for s in spectrum:
            otpt *= s
            
        return  np.fft.ifft(otpt).real
        
#         return np.fft.ifft(np.prod(scale * np.fft.fft(polynomials, n=new_length, norm=None, axis=1), 
#                                    axis=0), norm=None).real

In [487]:
polynomials = np.c_[np.ones_like(diffc), np.exp(-diffc)]

In [490]:
polynomials[-1]

array([1.      , 0.082085])

In [528]:
n_items = 60
diffc = np.linspace(-2.5, 2.5, n_items)

In [529]:
rr = _denominator(diffc)
rr

array([-4.50074555e+07, -3.06441208e+07, -7.87742334e+06, -7.98179221e+06,
       -1.46242406e+06,  3.72654016e+08,  7.72713229e+09,  1.25927911e+11,
        1.70335036e+12,  1.95018103e+13,  1.91317256e+14,  1.62340275e+15,
        1.20060835e+16,  7.78768293e+16,  4.45373374e+17,  2.25562981e+18,
        1.01547065e+19,  4.07673988e+19,  1.46351219e+20,  4.70913987e+20,
        1.36091168e+21,  3.53849912e+21,  8.29008232e+21,  1.75228080e+22,
        3.34520742e+22,  5.77310375e+22,  9.01331959e+22,  1.27381483e+23,
        1.63030891e+23,  1.89022489e+23,  1.98572427e+23,  1.89022489e+23,
        1.63030891e+23,  1.27381483e+23,  9.01331959e+22,  5.77310375e+22,
        3.34520742e+22,  1.75228080e+22,  8.29008232e+21,  3.53849912e+21,
        1.36091168e+21,  4.70913987e+20,  1.46351219e+20,  4.07673988e+19,
        1.01547065e+19,  2.25562981e+18,  4.45373373e+17,  7.78768293e+16,
        1.20060835e+16,  1.62340274e+15,  1.91317276e+14,  1.95018706e+13,
        1.70338936e+12,  

In [530]:
vg = np.c_[np.ones_like(diffc), np.exp(-diffc)]
g = 1

for ndx in range(diffc.size):
    g = np.convolve(g, vg[ndx])
    
g

array([1.00000000e+00, 1.49002560e+02, 1.06249299e+04, 4.83119899e+05,
       1.57489104e+07, 3.92335692e+08, 7.77535874e+09, 1.26004530e+11,
       1.70341958e+12, 1.95018803e+13, 1.91317294e+14, 1.62340277e+15,
       1.20060835e+16, 7.78768293e+16, 4.45373373e+17, 2.25562981e+18,
       1.01547065e+19, 4.07673988e+19, 1.46351219e+20, 4.70913987e+20,
       1.36091168e+21, 3.53849912e+21, 8.29008232e+21, 1.75228080e+22,
       3.34520742e+22, 5.77310375e+22, 9.01331959e+22, 1.27381483e+23,
       1.63030891e+23, 1.89022489e+23, 1.98572427e+23, 1.89022489e+23,
       1.63030891e+23, 1.27381483e+23, 9.01331959e+22, 5.77310375e+22,
       3.34520742e+22, 1.75228080e+22, 8.29008232e+21, 3.53849912e+21,
       1.36091168e+21, 4.70913987e+20, 1.46351219e+20, 4.07673988e+19,
       1.01547065e+19, 2.25562981e+18, 4.45373373e+17, 7.78768293e+16,
       1.20060835e+16, 1.62340277e+15, 1.91317294e+14, 1.95018803e+13,
       1.70341958e+12, 1.26004530e+11, 7.77535874e+09, 3.92335692e+08,
      

In [534]:
rr-g

array([-4.50074565e+07, -3.06442698e+07, -7.88804827e+06, -8.46491211e+06,
       -1.72113344e+07, -1.96816757e+07, -4.82264557e+07, -7.66188362e+07,
       -6.92177030e+07, -7.00015650e+07, -3.77331717e+07, -2.02409685e+07,
        1.05324200e+07,  4.15821600e+07,  5.98108160e+07,  6.47541760e+07,
        6.26995200e+07,  4.92912640e+07,  3.07200000e+07,  1.76291840e+07,
        2.07093760e+07,  3.25058560e+07,  7.65460480e+07,  9.43718400e+07,
        1.42606336e+08,  1.76160768e+08,  1.84549376e+08,  1.34217728e+08,
        1.67772160e+08,  1.34217728e+08, -3.35544320e+07, -6.71088640e+07,
       -1.00663296e+08, -1.67772160e+08, -2.01326592e+08, -1.42606336e+08,
       -5.87202560e+07,  4.19430400e+06,  5.97688320e+07,  1.38412032e+08,
        1.59383552e+08,  1.65609472e+08,  1.51207936e+08,  1.10592000e+08,
        6.05675520e+07,  1.26272000e+07, -1.56457600e+07, -2.72343360e+07,
       -2.81049660e+07, -2.36520892e+07, -1.78709456e+07, -9.73387961e+06,
       -3.02175351e+07, -

In [535]:
np.fft.ifft(g) - np.fft.ifft(rr)

array([ -8388608.               +0.j        ,
        25165824.         -7077888.j        ,
        16777216.         +9175040.j        ,
       -23068672.        -20971520.j        ,
         4194304.        +27262976.j        ,
        -5242880.        -31195136.j        ,
         -524288.         +3932160.j        ,
         3145728.          +524288.j        ,
         1703936.         +1114112.j        ,
         1081344.         -1228800.j        ,
          557056.          +679936.j        ,
         -274432.           +86016.j        ,
         -549888.          -309248.j        ,
        -1100032.          +343808.j        ,
         1925312.           -85952.j        ,
        -1925264.          +360976.j        ,
         4400582.          -773540.j        ,
        -3300436.          +601642.j        ,
         4400581.25        -618831.75j      ,
        -2200290.62109375  +550072.65625j   ,
         2200290.62304688  -962627.14794922j,
               0.         +1272043

In [532]:
np.fft.ifft(rr)

array([ 2.59482185e+22-0.00000000e+00j, -2.45645048e+22+1.26622903e+21j,
        2.08388164e+22-2.15408479e+21j, -1.58377521e+22+2.46666396e+21j,
        1.07792280e+22-2.25254181e+21j, -6.56599976e+21+1.72918570e+21j,
        3.57689313e+21-1.14187052e+21j, -1.74101777e+21+6.56339224e+20j,
        7.56345780e+20-3.30542436e+20j, -2.92895765e+20+1.46398848e+20j,
        1.00963621e+20-5.71415370e+19j, -3.09305612e+19+1.96738508e+19j,
        8.40663789e+18-5.97645889e+18j, -2.02319640e+18+1.60130487e+18j,
        4.30262339e+17-3.78147412e+17j, -8.06742774e+16+7.86231492e+16j,
        1.33044577e+16-1.43741058e+16j, -1.92485231e+15+2.30732469e+15j,
        2.43630444e+14-3.24659372e+14j, -2.68969164e+13+3.99749367e+13j,
        2.58167635e+12-4.29941697e+12j, -2.14668054e+11+4.03183007e+11j,
        1.54042347e+10-3.29057588e+10j, -9.43924677e+08+2.33272061e+09j,
        5.17068296e+07-1.42950131e+08j,  2.20029062e+06+7.42598085e+06j,
        1.10014531e+06-2.75036328e+05j,  2.20029062